In [1]:
%run init.ipynb

Using TensorFlow backend.


matchzoo version 2.1.0

data loading ...
data loaded as `train_pack_raw` `dev_pack_raw` `test_pack_raw`
`ranking_task` initialized with metrics [normalized_discounted_cumulative_gain@3(0.0), normalized_discounted_cumulative_gain@5(0.0), mean_average_precision(0.0)]
loading embedding ...
embedding loaded as `glove_embedding`


In [2]:
preprocessor = mz.preprocessors.BasicPreprocessor(fixed_length_left=10, 
                                                  fixed_length_right=100, 
                                                  remove_stop_words=False)

In [3]:
train_pack_processed = preprocessor.fit_transform(train_pack_raw, drop_invalid=True)
dev_pack_processed = preprocessor.transform(dev_pack_raw, drop_invalid=False)
test_pack_processed = preprocessor.transform(test_pack_raw, drop_invalid=False)

Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3790.79it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2343.40it/s]
Building Vocabulary from a datapack.: 100%|██████████| 404415/404415 [00:00<00:00, 1775711.42it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 2118/2118 [00:00<00:00, 3787.75it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 18841/18841 [00:08<00:00, 2350.14it/s]
Processing text_right with transform: 100%|██████████| 18841/18841 [00:00<00:00, 32494.44it/s]
Processing text_left with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 122/122 [00:00<00:00, 3433.39it/s]
Processing text_right with chain_transform of Tokenize => Lowercase => PuncRemoval: 100%|██████████| 1115/1115 [00:00

In [4]:
preprocessor.context

{'filter_unit': <matchzoo.preprocessors.units.frequency_filter.FrequencyFilter at 0x7f2c2804eb00>,
 'vocab_unit': <matchzoo.preprocessors.units.vocabulary.Vocabulary at 0x7f2bb11b55c0>,
 'vocab_size': 16675,
 'embedding_input_dim': 16675,
 'input_shapes': [(10,), (100,)]}

In [5]:
model = mz.models.ArcII()

# load `input_shapes` and `embedding_input_dim` (vocab_size)
model.params.update(preprocessor.context)

model.params['task'] = ranking_task
model.params['embedding_output_dim'] = glove_embedding.output_dim
model.params['embedding_trainable'] = True
model.params['num_blocks'] = 2
model.params['kernel_1d_count'] = 32
model.params['kernel_1d_size'] = 3
model.params['kernel_2d_count'] = [64, 64]
model.params['kernel_2d_size'] = [3, 3]
model.params['pool_2d_size'] = [[3, 3], [3, 3]]
model.params['optimizer'] = 'adam'

model.build()
model.compile()

print(model.params)

model_class                   <class 'matchzoo.models.arcii.ArcII'>
input_shapes                  [(10,), (100,)]
task                          Ranking Task
optimizer                     adam
with_embedding                True
embedding_input_dim           16675
embedding_output_dim          300
embedding_trainable           True
num_blocks                    2
kernel_1d_count               32
kernel_1d_size                3
kernel_2d_count               [64, 64]
kernel_2d_size                [3, 3]
activation                    relu
pool_2d_size                  [[3, 3], [3, 3]]
padding                       same
dropout_rate                  0.0


In [6]:
model.backend.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
text_left (InputLayer)          (None, 10)           0                                            
__________________________________________________________________________________________________
text_right (InputLayer)         (None, 100)          0                                            
__________________________________________________________________________________________________
embedding (Embedding)           multiple             5002500     text_left[0][0]                  
                                                                 text_right[0][0]                 
__________________________________________________________________________________________________
conv1d_1 (Conv1D)               (None, 10, 32)       28832       embedding[0][0]                  
__________

In [7]:
embedding_matrix = glove_embedding.build_matrix(preprocessor.context['vocab_unit'].state['term_index'])

In [8]:
model.load_embedding_matrix(embedding_matrix)

In [9]:
test_x, test_y = test_pack_processed[:].unpack()
evaluate = mz.callbacks.EvaluateAllMetrics(model, x=test_x, y=test_y, batch_size=len(test_y))

In [10]:
train_generator = mz.DataGenerator(
    train_pack_processed,
    mode='pair',
    num_dup=2,
    num_neg=1,
    batch_size=20
)
print('num batches:', len(train_generator))

num batches: 102


In [11]:
history = model.fit_generator(train_generator, 
                              epochs=30, 
                              callbacks=[evaluate], 
                              workers=30, 
                              use_multiprocessing=True)

Epoch 1/30
102/102 [==============================] - 24s 233ms/step - loss: 0.5510
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5654536022684477 - normalized_discounted_cumulative_gain@5(0.0): 0.6388633228950963 - mean_average_precision(0.0): 0.5870044236946693
Epoch 2/30
102/102 [==============================] - 25s 248ms/step - loss: 0.2229
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5947598744683228 - normalized_discounted_cumulative_gain@5(0.0): 0.652043351860647 - mean_average_precision(0.0): 0.6056211957528971
Epoch 3/30
102/102 [==============================] - 24s 238ms/step - loss: 0.1027
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5641112770025778 - normalized_discounted_cumulative_gain@5(0.0): 0.617637113397053 - mean_average_precision(0.0): 0.5729277048917536
Epoch 4/30
102/102 [==============================] - 24s 234ms/step - loss: 0.0674
Validation: normalized_discounted_cumulative_gain@3(0.0): 0.5515358717237778 - norma